# PTST Idealization using axi-symmetric 2D interface layer

## Purpose of the notebook

Verify the axi-symmetric model in an elastic state to ensure that the confinement is
correctly reproduced. This notebook serves as a basis for 3D degradation within
a ligament/bond zone in axi-symmetric models. Examples are
 - PTST
 - prestressing
 - confined bond configuration
 - effect of splitting cracks
 - restrained splitting cracks in fatigue tests

## Implementation remarks

- The notebook now uses `BCSliceI` - to ensure the same order of nodes along an 
interfaces
- Compression is applied using individual `BCDof` instances without integration 
  over the boundary. This should be changed to BCSliceE - which can use the 
  finite element shape functions to integrate over the element domain

# Elastic material model for the ligament

 - check the uniformity of the radial stress 

In [61]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
import warnings
import matplotlib.pylab as plt
import numpy as np
from ibvpy.api import TStepBC, TFCyclicNonsymmetricConstant, TFBilinear
from ibvpy.bcond import BCSliceI, BCDof
from ibvpy.xmodel.xdomain_fe_grid_axisym import XDomainFEGridAxiSym
from ibvpy.fets import FETS2D4Q
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic
from ibvpy.tmodel.mats2D.mats2D_elastic.vmats2D_elastic import \
    MATS2DElastic

## FE Axisymmetric discretization

In [77]:
n_x_e = 20
n_inner_y_e = 10
n_outer_y_e = 10
L_x = 100.0 # [mm]
d_x = L_x / n_x_e
R_in = 25 # [mm]
dR = 10 # [mm]
R_out = 50 # [mm]
len_notch = 10
n_notch = 2
# geo_transform
# gmesh
xd_inner = XDomainFEGridAxiSym(integ_factor = 2 * np.pi,
                               coord_min=(0, 0),
                               coord_max=(L_x, R_in),
                               shape=(n_x_e, n_inner_y_e),
                               fets=FETS2D4Q())
xd_middle = XDomainFEGridAxiSym(integ_factor = 2 * np.pi,
                               coord_min=(0+n_notch*d_x, R_in),
                               coord_max=(L_x-n_notch*d_x, R_in+dR),
                               shape=(n_x_e-2*n_notch, 1),
                               fets=FETS2D4Q())
xd_outer = XDomainFEGridAxiSym(integ_factor = 2 * np.pi,
                               coord_min=(0, R_in+dR),
                               coord_max=(L_x, R_out),
                               shape=(n_x_e, n_outer_y_e),
                               fets=FETS2D4Q())
m_elastic = MATS3DElastic(E=37000, nu=0.18)
m_inelastic = MATS3DElastic(E=37000, nu=0.18)
m = TStepBC(
    domains=[(xd_inner, m_elastic),
             (xd_outer, m_elastic),
             (xd_middle, m_inelastic),
             ]
)

## Boundary and transition conditions

### Kinematic links between the domains

In [78]:
link_inner_middle = BCSliceI(var='u', dims=[0, 1],
                             slice=xd_inner.mesh.I[n_notch:-n_notch, -1], 
                             link_slice=xd_middle.mesh.I[:, 0],
                             link_coeffs=[1.0, 1.0],
                             link_dims=[0, 1],
                             value=0.0
                            )
link_middle_outer = BCSliceI(slice=xd_outer.mesh.I[n_notch:-n_notch, 0], 
                             var='u', dims=[0, 1],
                             link_slice=xd_middle.mesh.I[:, -1],
                             link_coeffs=[1, 1],
                             link_dims=[0, 1],
                             value=0,
                            )

### Compression/Traction fc

In [79]:
inner_fixed_1 = BCSliceI(slice=xd_inner.mesh.I[:, 0], var='u', dims=[1], value=0)
outer_fixed_0 = BCSliceI(slice=xd_outer.mesh.I[0, :], var='u', dims=[0], value=0)

outer_compression_slice = BCSliceI(slice=xd_outer.mesh.I[:, -1],
                                   var='u', dims=[1], value=0.01)

compression_dofs = outer_compression_slice.dofs
compression = -10000*L_x / len(compression_dofs)
outer_compression_force_first = [BCDof(var='f', dof=dof, value=compression) 
                                 for dof in compression_dofs ]
outer_compression_force_first[0].value *=0.5
outer_compression_force_first[-1].value *=0.5
bc1 = [inner_fixed_1, 
       outer_fixed_0,
       link_inner_middle,
       link_middle_outer
      ] + outer_compression_force_first

### Sliding dc

In [80]:
#lower_fixed_0 = BCSlice(slice=xd_lower.mesh[:, 0, :, 0], var='u', dims=[1], value=0)
#upper_fixed_1 = BCSlice(slice=xd_upper.mesh[0, :, 0, :], var='u', dims=[0], value=0)
#
#lower_slide = BCSlice(slice=xd_lower.mesh[0, :, 0, :], var='u', dims=[0], value=6)
#
#
#bc1 =   [lower_fixed_0, upper_fixed_1, lower_slide]

### Sliding fc

In [81]:
#lower_fixed_0 = BCSlice(slice=xd_lower.mesh[:, 0, :, 0], var='u', dims=[1], value=0)
#upper_fixed_1 = BCSlice(slice=xd_upper.mesh[0, :, 0, :], var='u', dims=[0], value=0)
#
#
#
#lower_slide = BCSlice(slice=xd_lower.mesh[0, :, 0, :], var='u', dims=[0], value=0)
#Force = 10 / len(lower_slide.dofs)
#lower_slide_force = [BCDof(var='f', dof=dof, value = Force) 
#             for dof in lower_slide.dofs]
#
#bc1 =   [lower_fixed_0, upper_fixed_1] + lower_slide_force

In [82]:
m.bc=bc1
m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 0.1
# Accessing the following property - ensures serialization
s.tstep.fe_domain.serialized_subdomains

In [83]:
s.reset()
s.run()

t:	 0.00(0), 
	 0.10(1), 
	 0.20(1), 
	 0.30(1), 
	 0.40(1), 
	 0.50(1), 
	 0.60(1), 
	 0.70(1), 
	 0.80(1), 
	 0.90(1), 
	 1.00(1), 



## Postprocesing

### Access the last displacement step

In [84]:
U_last = m.hist.U_t[-1]
F_last = m.hist.U_t[-1]

In [85]:
#U_last[xd_middle.o_Ia], U_last[xd_inner.o_Ia]

In [86]:
#xd_middle.map_U_to_field(U_last)

In [88]:
if False:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    strain_viz = Viz3DTensorField(vis3d=m.hist['stress'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()

# Slide34 model used for a ligament

 - check the correctness of the wrapper interface  

In [89]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
import warnings
import matplotlib.pylab as plt
import numpy as np
from ibvpy.api import TStepBC, TFCyclicNonsymmetricConstant, TFBilinear
from ibvpy.bcond import BCSliceI, BCDof
from ibvpy.xmodel.xdomain_fe_grid_axisym import XDomainFEGridAxiSym
from ibvpy.fets import FETS2D4Q
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic
from ibvpy.tmodel.mats2D.mats2D_elastic.vmats2D_elastic import \
    MATS2DElastic
from bmcs_matmod.slide.vslide_34_ij import MATS3DSlideStrain, Slide34

In [117]:
from bmcs_matmod.slide.vslide_34_ij import MATS3DSlideStrain
class Debugging(MATS3DSlideStrain):
    def get_corr_pred(self, eps_Emab_n1, tn1, **state):
        r'''
        Corrector predictor computation.
        '''
        n_i = self.n_a
        eps_ij = eps_Emab_n1
        eps_N = np.einsum('...ij,...i,...j->...', eps_ij, n_i, n_i)
        #
        # Change #1
        # Instead of concatenation I reuse the eps_NT_Ema - in-place assuming that the first eps_NT_Ema[...,0] is zero 
        # Check if it is true
        #
        # eps_T = self.get_eps_T(eps_ij, n_i)
        # eps_T = np.sqrt(np.einsum('...i,...i->...',eps_T, eps_T))
        eps_NT_Ema = self.get_eps_T(eps_ij, n_i)
        eps_NT_Ema[..., 0] = eps_N
        # print(eps_N.shape, eps_T)
        # eps_NT_Ema = np.concatenate([np.transpose(eps_N), np.transpose(eps_T)], axis=-1)
        # print('eps_NT_Ema', eps_NT_Ema.shape)
        # print(self.state_var_shapes)
        se = self.slide_displ
        sig_NT_Ema, D_Emab = se.get_corr_pred(eps_NT_Ema, tn1, **state)
        
        eps_N_p, eps_T_p_x, eps_T_p_y = state['w_pi'], state['s_pi_x'], state['s_pi_y']
        eps_T = self.get_eps_T(eps_ij, n_i)
        eps_T_p_i = self.get_eps_T_p(eps_T_p_x, eps_T)
        omega_N_Em, omega_T_Em = state['omega_N'], state['omega_T']
        phi_Emab = np.zeros_like(eps_Emab_n1)
        # Change #2
        # The ellipsis index must be used to handle the element, material point dimensions
        #         phi_Emab[:,1, 1] = 0.
        #         phi_Emab[:,2, 2] = np.sqrt(1 - omega_T_Em)
        #         phi_Emab[:,0, 0] = np.sqrt(1 - omega_N_Em)
        phi_Emab[..., 0, 0] = np.sqrt(1 - omega_N_Em)
        phi_Emab[..., 1, 1] = np.sqrt(1 - omega_T_Em)
        #
        # Change #3
        # - hoop stress - elastic - integrity - not damage
        # phi_Emab[..., 2, 2] = 0
        phi_Emab[..., 2, 2] = 1

        # Change #3
        #
        # I would like to avoid transpose in the code - it should 
        # stick to index based notation. We do not write eps_ij^T in formulas.
        #
        #         beta_Emijkl = np.einsum('...ik,...jl->...ijkl', phi_Emab,
        #                               np.transpose(phi_Emab, (1, 0)))
        beta_Emijkl = np.einsum('...ik,...lj->...ijkl', phi_Emab, phi_Emab)

        eps_ij_p = (np.einsum('i,...j->...ij', n_i, eps_T_p_i) +
                    np.einsum('...i,j->...ij', eps_T_p_i,n_i) +
                    np.einsum('...,i,j->...ij', eps_N_p, n_i, n_i)
                   )

        D_abef = self.D_abef
#         print('D_abef', D_abef)
        D_Emabcd = np.einsum('...ijkl,klrs,...rstu->...ijtu', beta_Emijkl, D_abef, beta_Emijkl)
#         print('D_abef', D_Emabcd)

        sigma_Emab = np.einsum('...ijkl,...kl->...ij', D_Emabcd, (eps_Emab_n1 - eps_ij_p))

        return sigma_Emab, D_Emabcd

## FE Axisymmetric discretization

In [118]:
n_x_e = 1
n_inner_y_e = 1
n_outer_y_e = 1
L_x = 100.0 # [mm]
d_x = L_x / n_x_e
R_in = 25 # [mm]
dR = 10 # [mm]
R_out = 50 # [mm]
len_notch = 10
n_notch = 0
# geo_transform
# gmesh
xd_inner = XDomainFEGridAxiSym(integ_factor = 2 * np.pi,
                               coord_min=(0, 0),
                               coord_max=(L_x, R_in),
                               shape=(n_x_e, n_inner_y_e),
                               fets=FETS2D4Q())
xd_middle = XDomainFEGridAxiSym(integ_factor = 2 * np.pi,
                               coord_min=(0+n_notch*d_x, R_in),
                               coord_max=(L_x-n_notch*d_x, R_in+dR),
                               shape=(n_x_e-2*n_notch, 1),
                               fets=FETS2D4Q())
xd_outer = XDomainFEGridAxiSym(integ_factor = 2 * np.pi,
                               coord_min=(0, R_in+dR),
                               coord_max=(L_x, R_out),
                               shape=(n_x_e, n_outer_y_e),
                               fets=FETS2D4Q())
m_elastic = MATS3DElastic(E=37000, nu=0.18)
material_params =  dict(
     E_T=37000, gamma_T=500, K_T=0, S_T=1000000.5, c_T=6, bartau=300, 
     E_N=37000, S_N=1000000.5, c_N = 3, m = 0.05, f_t=2, f_c=6000, f_c0 = 4000, eta=0.)
bond_m = Slide34(**material_params)
#m_inelastic = MATS3DSlideStrain(slide_displ=bond_m)
m_inelastic = Debugging(slide_displ=bond_m)
m = TStepBC(
    domains=[(xd_inner, m_elastic),
             (xd_outer, m_elastic),
             (xd_middle, m_inelastic),
             ]
)

## Boundary and transition conditions

### Kinematic links between the domains

In [119]:
link_inner_middle = BCSliceI(var='u', dims=[0, 1],
                             slice=xd_inner.mesh.I[:, -1], 
#                             slice=xd_inner.mesh.I[n_notch:-n_notch, -1], 
                             link_slice=xd_middle.mesh.I[:, 0],
                             link_coeffs=[1.0, 1.0],
                             link_dims=[0, 1],
                             value=0.0
                            )
link_middle_outer = BCSliceI(var='u', dims=[0, 1],
#                              slice=xd_outer.mesh.I[n_notch:-n_notch, 0], 
                             slice=xd_outer.mesh.I[:, 0], 
                             link_slice=xd_middle.mesh.I[:, -1],
                             link_coeffs=[1, 1],
                             link_dims=[0, 1],
                             value=0,
                            )

### Compression/Traction fc

In [120]:
inner_fixed_1 = BCSliceI(slice=xd_inner.mesh.I[:, 0], var='u', dims=[1], value=0)
outer_fixed_0 = BCSliceI(slice=xd_outer.mesh.I[0, :], var='u', dims=[0], value=0)

outer_compression_slice = BCSliceI(slice=xd_outer.mesh.I[:, -1],
                                   var='u', dims=[1], value=0.01)

compression_dofs = outer_compression_slice.dofs
compression = -10000*L_x / len(compression_dofs)
outer_compression_force_first = [BCDof(var='f', dof=dof, value=compression) 
                                 for dof in compression_dofs ]
outer_compression_force_first[0].value *=0.5
outer_compression_force_first[-1].value *=0.5
bc1 = [inner_fixed_1, 
#       outer_fixed_0,
       link_inner_middle,
       link_middle_outer
      ] + outer_compression_force_first

### Sliding dc

In [121]:
#lower_fixed_0 = BCSlice(slice=xd_lower.mesh[:, 0, :, 0], var='u', dims=[1], value=0)
#upper_fixed_1 = BCSlice(slice=xd_upper.mesh[0, :, 0, :], var='u', dims=[0], value=0)
#
#lower_slide = BCSlice(slice=xd_lower.mesh[0, :, 0, :], var='u', dims=[0], value=6)
#
#
#bc1 =   [lower_fixed_0, upper_fixed_1, lower_slide]

### Sliding fc

In [122]:
#lower_fixed_0 = BCSlice(slice=xd_lower.mesh[:, 0, :, 0], var='u', dims=[1], value=0)
#upper_fixed_1 = BCSlice(slice=xd_upper.mesh[0, :, 0, :], var='u', dims=[0], value=0)
#
#
#
#lower_slide = BCSlice(slice=xd_lower.mesh[0, :, 0, :], var='u', dims=[0], value=0)
#Force = 10 / len(lower_slide.dofs)
#lower_slide_force = [BCDof(var='f', dof=dof, value = Force) 
#             for dof in lower_slide.dofs]
#
#bc1 =   [lower_fixed_0, upper_fixed_1] + lower_slide_force

In [123]:
m.bc=bc1
m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 0.1
# Accessing the following property - ensures serialization
s.tstep.fe_domain.serialized_subdomains

In [124]:
s.reset()
s.run()

t:	 0.00u_N, u_T [[0. 0. 0. 0.]] [[0. 0. 0. 0.]] [[0. 0. 0. 0.]]
(0), 
u_N, u_T [[0. 0. 0. 0.]] [[0. 0. 0. 0.]] [[0. 0. 0. 0.]]
	 0.10u_N, u_T [[0. 0. 0. 0.]] [[0. 0. 0. 0.]] [[0. 0. 0. 0.]]
u_N, u_T [[-3.71908529e-05 -3.71908529e-05 -3.71908529e-05 -3.71908529e-05]] [[0. 0. 0. 0.]] [[0. 0. 0. 0.]]
(1), 
u_N, u_T [[-3.71908529e-05 -3.71908529e-05 -3.71908529e-05 -3.71908529e-05]] [[0. 0. 0. 0.]] [[0. 0. 0. 0.]]
	 0.20u_N, u_T [[-3.71908529e-05 -3.71908529e-05 -3.71908529e-05 -3.71908529e-05]] [[0. 0. 0. 0.]] [[0. 0. 0. 0.]]
u_N, u_T [[-7.43817058e-05 -7.43817058e-05 -7.43817058e-05 -7.43817058e-05]] [[0. 0. 0. 0.]] [[0. 0. 0. 0.]]
(1), 
u_N, u_T [[-7.43817058e-05 -7.43817058e-05 -7.43817058e-05 -7.43817058e-05]] [[0. 0. 0. 0.]] [[0. 0. 0. 0.]]
	 0.30u_N, u_T [[-7.43817058e-05 -7.43817058e-05 -7.43817058e-05 -7.43817058e-05]] [[0. 0. 0. 0.]] [[0. 0. 0. 0.]]
u_N, u_T [[-0.00011157 -0.00011157 -0.00011157 -0.00011157]] [[0. 0. 0. 0.]] [[0. 0. 0. 0.]]
(1), 
u_N, u_T [[-0.00011157 -0.000111

## Postprocesing

### Access the last displacement step

In [111]:
U_last = m.hist.U_t[-1]

In [112]:
#U_last[xd_middle.o_Ia], U_last[xd_inner.o_Ia]

In [140]:
eps_Emab = xd_outer.map_U_to_field(U_last)

In [142]:
m_inelastic.get_corr_pred(eps_Emab, 1, **m.fe_domain[2].state_k)

u_N, u_T [[-0.00035219 -0.00035219 -0.00035219 -0.00035219]] [[0. 0. 0. 0.]] [[0. 0. 0. 0.]]


(array([[[[ -0.59082362,   0.05357139,   0.        ],
          [  0.05357139, -15.16668365,   0.        ],
          [  0.        ,   0.        , -15.37107986]],
 
         [[ -0.59082362,  -0.05357139,   0.        ],
          [ -0.05357139, -15.16668365,   0.        ],
          [  0.        ,   0.        , -15.37107986]],
 
         [[ -0.62108113,  -0.05357139,   0.        ],
          [ -0.05357139, -15.16479833,   0.        ],
          [  0.        ,   0.        , -15.33139573]],
 
         [[ -0.62108113,   0.05357139,   0.        ],
          [  0.05357139, -15.16479833,   0.        ],
          [  0.        ,   0.        , -15.33139573]]]]),
 array([[[[[[37777.77777778,     0.        ,     0.        ],
            [    0.        ,  9444.44444444,     0.        ],
            [    0.        ,     0.        ,  9444.44444444]],
 
           [[    0.        , 14166.66666667,     0.        ],
            [14166.66666667,     0.        ,     0.        ],
            [    0.       

In [139]:
m.fe_domain[2].get_corr_pred(U_last,1, 1)

u_N, u_T [[-0.00037191 -0.00037191 -0.00037191 -0.00037191]] [[0. 0. 0. 0.]] [[0. 0. 0. 0.]]


(array([    488.27466054,  125488.64786776,    -488.27466054,
         125488.64786776,    -899.79665829, -174168.46804043,
            899.79665829, -174168.46804043]),
 array([16, 17, 20, 21, 22, 23, 18, 19]))

In [114]:
if True:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    strain_viz = Viz3DTensorField(vis3d=m.hist['stress'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()